In [16]:
import numpy as np
import random
import itertools 
import os
from collections import defaultdict
import json
import pandas as pd

In [6]:
import random
import numpy as np
from time import time
from tqdm import tqdm

In [13]:
import numpy as np
from tqdm import tqdm

red = ['0'] * 26
black = ['1'] * 26
deck = black + red

np.random.seed(123)

results = []
for i in tqdm(range(10000)):
    np.random.shuffle(deck)
    results += [int(''.join(deck),2)]

#np.save('results_10000', results)

100%|█████████████████████████████████| 10000/10000 [00:00<00:00, 186786.25it/s]


## Reading in Npy 

In [7]:
def load_process_simulations(file_path): 
    simulations = np.load(file_path)
    
    def int_to_bit(n):
        binary = bin(n)[2:] #what? 
        return binary.zfill(52)
    
    binary_simulations = [int_to_bit(sim) for sim in simulations]             
    
    return binary_simulations             

In [4]:
#processed_sims = load_process_simulations('./penney-game/simulation/results.npy')


In [8]:
processed_sims = load_process_simulations('results_10000.npy')


In [11]:
short_processed_sims = processed_sims[:50]
short_processed_sims

['0001111101110110011000010011011011010100100101100010',
 '0101010100111000001101100100010101011000101101101111',
 '0000001110111001101100010010000000111110011110111011',
 '1010001100001111011010010001111011010001101101100001',
 '0011000111000101100110011001100011111100101001110010',
 '1111010101100101010101000111111100000110001001000101',
 '0111010000110000000111001110100011101101001101011101',
 '0000100111110001100010001110001111110111100100101100',
 '0001101010101111001001100001101111010110111010010000',
 '0001101011001111110101011100001111001100001000011001',
 '1010100110101001011001100011100101100011111010010100',
 '1111100110101100010101011100110011011000101000001001',
 '0001011010010111110101001100000011111111010001000110',
 '1100001000100110100110110101011100010011011011110010',
 '1110111110010011001011000000101010100000101011110101',
 '1000100111000110110001101100010000111101011001010111',
 '0100001101000100101100011100110101111111100011010010',
 '11101000110011101011100100001

In [9]:
def variation1(game, player1_sequence, player2_sequence):
    player1_cards = 0
    player2_cards = 0
    pile = 0
    
    for i in range(len(game) - 2):
        current_sequence = game[i:i+3]
        pile += 1
        if current_sequence == player1_sequence:
            player1_cards += pile
            pile = 0
        elif current_sequence == player2_sequence:
            player2_cards += pile
            pile = 0
    
    return player1_cards, player2_cards

In [13]:
def analyze_all_combinations(processed_sims):
    all_sequences = [''.join(seq) for seq in itertools.product('01', repeat=3)]
    results = {(p1, p2): {'player1_wins': 0, 'player2_wins': 0}
               for p1 in all_sequences for p2 in all_sequences if p1 != p2}
    
    for game in processed_sims:
        for (p1_seq, p2_seq), result in results.items():
            p1_cards, p2_cards = variation1(game, p1_seq, p2_seq)
            if p1_cards > p2_cards:
                result['player1_wins'] += 1
            elif p2_cards > p1_cards:
                result['player2_wins'] += 1
    
    return results

In [14]:
%%time
results = analyze_all_combinations(short_processed_sims)
results

CPU times: user 47.3 ms, sys: 1.93 ms, total: 49.3 ms
Wall time: 48.9 ms


{('000', '001'): {'player1_wins': 20, 'player2_wins': 30},
 ('000', '010'): {'player1_wins': 18, 'player2_wins': 30},
 ('000', '011'): {'player1_wins': 3, 'player2_wins': 46},
 ('000', '100'): {'player1_wins': 0, 'player2_wins': 50},
 ('000', '101'): {'player1_wins': 14, 'player2_wins': 35},
 ('000', '110'): {'player1_wins': 1, 'player2_wins': 49},
 ('000', '111'): {'player1_wins': 28, 'player2_wins': 22},
 ('001', '000'): {'player1_wins': 30, 'player2_wins': 20},
 ('001', '010'): {'player1_wins': 45, 'player2_wins': 5},
 ('001', '011'): {'player1_wins': 46, 'player2_wins': 2},
 ('001', '100'): {'player1_wins': 0, 'player2_wins': 50},
 ('001', '101'): {'player1_wins': 42, 'player2_wins': 7},
 ('001', '110'): {'player1_wins': 18, 'player2_wins': 28},
 ('001', '111'): {'player1_wins': 49, 'player2_wins': 1},
 ('010', '000'): {'player1_wins': 30, 'player2_wins': 18},
 ('010', '001'): {'player1_wins': 5, 'player2_wins': 45},
 ('010', '011'): {'player1_wins': 23, 'player2_wins': 23},
 ('010

In [25]:
df = pd.DataFrame.from_dict(results, orient = 'index')
df.reset_index(inplace=True)
df.columns = ['Sequence 1','Sequence 2', 'Player 1 Wins', 'Player 2 Wins']
df

,Sequence 1,Sequence 2,Player 1 Wins,Player 2 Wins
0,000,001,20,30
1,000,010,18,30
2,000,011,3,46
3,000,100,0,50
4,000,101,14,35
5,000,110,1,49
6,000,111,28,22
7,001,000,30,20
8,001,010,45,5
9,001,011,46,2


In [ ]:
#i in range length of data frame 
#convert 

In [20]:
#string_keyed_results = {str(key): value for key, value in results.items()}
